# Global Tweet Sentiment Analysis - Mood of the World { Free API Version }

# Install Required Packages

In [ ]:
!pip install tweepy textblob plotly pandas numpy wordcloud matplotlib seaborn python-dotenv scikit-learn --quiet
!python -m textblob.download_corpora

# Import Libraries

In [ ]:
import tweepy
import pandas as pd
import numpy as np
from textblob import TextBlob
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import re
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pyo.init_notebook_mode(connected=True)

# Twitter API Configuration
## Note : get API from https://developer.twitter.com/


In [ ]:
class TwitterConfig:
    def __init__(self):
        # Replace with your actual API credentials
        self.API_KEY = "YOUR_API_KEY"
        self.API_SECRET = "YOUR_API_SECRET"
        self.ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
        self.ACCESS_TOKEN_SECRET = "YOUR_ACCESS_TOKEN_SECRET"
        self.BEARER_TOKEN = "YOUR_BEARER_TOKEN"

        # Rate limit settings
        self.FREE_TIER_MONTHLY_LIMIT = 100
        self.DAILY_BUDGET = 10
        self.BATCH_SIZE = 25
        self.REQUEST_DELAY = 1.5

    def get_api_v1(self):
        """Get Twitter API v1.1 client"""
        auth = tweepy.OAuthHandler(self.API_KEY, self.API_SECRET)
        auth.set_access_token(self.ACCESS_TOKEN, self.ACCESS_TOKEN_SECRET)
        return tweepy.API(auth, wait_on_rate_limit=True)

    def get_api_v2(self):
        """Get Twitter API v2 client"""
        return tweepy.Client(
            bearer_token=self.BEARER_TOKEN,
            consumer_key=self.API_KEY,
            consumer_secret=self.API_SECRET,
            access_token=self.ACCESS_TOKEN,
            access_token_secret=self.ACCESS_TOKEN_SECRET,
            wait_on_rate_limit=True
        )
config = TwitterConfig()